In [2]:
import numpy as np
import json
import os
import pickle
from collections import OrderedDict
from utils import mean_dice, total_intersect_and_union

In [3]:
def load_pickle(path):
    outputs = []
    with open(path, "rb") as fin:
        pickler = pickle.Unpickler(fin)
        while True:
            try:
                item = pickler.load()
                outputs.append(item)
            except EOFError:
                break
    return outputs

dumps = load_pickle('dumps/val-20220226-181455.pkl') # dumps/test-20220226-174222.pkl')

In [8]:
def get_groups(json_path):
    data = json.load(open(json_path))
    print(data.keys())
    test = data["local_test"]
    idxs = [item['label'].split('/')[-1].replace('.nii.gz', '').split('_')[1:3] for item in test]
    idxs = [(int(idx[0]), int(idx[1])) for idx in idxs]
    group = OrderedDict()
    for idx in idxs:
        patientid, sliceid = idx
        if patientid not in group:
            group[patientid] = []
        group[patientid].append(sliceid)
    group_new = OrderedDict()
    for key in group: group_new[key] = (min(group[key]), max(group[key]))
    return group_new

groups = get_groups('/sailhome/yuhuiz/develop/data/MedicalImages/msd/processed/Task07_Pancreas/dataset_5slices.json')
print()

dict_keys(['training', 'validation', 'local_test', 'test', 'labels'])


In [5]:
dices = []
curidx = 0
for key in groups:
    dumps_ = dumps[groups[key][0]+curidx:groups[key][1]+curidx+1]
    curidx += groups[key][1]-groups[key][0]+1
    preds, labels = np.array([dump['logit'].argmax(0) for dump in dumps_]), np.array([dump['label'] for dump in dumps_])
    dices.append(total_intersect_and_union(preds, labels, 3, 255))

In [6]:
dices_1, dices_2 = [], []
for idx in range(len(dices)):
    item = dices[idx][0] * 2 / (dices[idx][2] + dices[idx][3])
    dices_1.append(item[1])
    dices_2.append(item[2])
    print(dices[idx])
print(np.mean(dices_1), np.mean(dices_2))

(array([2.9557511e+07, 3.3804000e+04, 2.5520000e+03]), array([2.9581197e+07, 6.1757000e+04, 7.7230000e+03]), array([2.9576201e+07, 4.3278000e+04, 2.7930000e+03]), array([2.9562507e+07, 5.2283000e+04, 7.4820000e+03]))
(array([2.3242768e+07, 4.6593000e+04, 5.7020000e+03]), array([2.3276373e+07, 7.7296000e+04, 1.2900000e+04]), array([2.3250295e+07, 7.1210000e+04, 9.3110000e+03]), array([2.3268846e+07, 5.2679000e+04, 9.2910000e+03]))
(array([1.0218859e+07, 2.2850000e+03, 0.0000000e+00]), array([1.0221331e+07, 4.7570000e+03, 0.0000000e+00]), array([1.0221323e+07, 2.2930000e+03, 0.0000000e+00]), array([1.0218867e+07, 4.7490000e+03, 0.0000000e+00]))
(array([2.5528126e+07, 7.7405000e+04, 2.2408000e+04]), array([25581208.,   137006.,    34071.]), array([2.5571536e+07, 9.3453000e+04, 2.5123000e+04]), array([25537798.,   120958.,    31356.]))
(array([2.3287893e+07, 3.1698000e+04, 6.7900000e+02]), array([2.3296797e+07, 4.2201000e+04, 2.3640000e+03]), array([2.329488e+07, 3.428300e+04, 1.653000e+03

/tmp/ipykernel_3985102/291321342.py:3: RuntimeWarning: invalid value encountered in true_divide
  item = dices[idx][0] * 2 / (dices[idx][2] + dices[idx][3])


In [7]:
dices_1

[0.707485271188037,
 0.7521733164364875,
 0.6489633626810565,
 0.7220245229955553,
 0.8578735842162952,
 0.7159443361787391,
 0.8203932022659114,
 0.7848780487804878,
 0.8511953690303907,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.7367193006619083]

In [45]:
dices_2

[0.2740740740740741,
 0.7711388738004707,
 0.5607951918631531,
 0.7273358805140674,
 0.0,
 0.5547780199609957,
 0.7001854925314849,
 0.6069100391134289,
 0.7546897546897547,
 0.0,
 0.0,
 0.5233926954421974,
 0.2573953131002689,
 0.1035002162468495,
 0.0]

In [42]:
np.array([0])/ np.array([0])

/tmp/ipykernel_3910573/2013172540.py:1: RuntimeWarning: invalid value encountered in true_divide
  np.array([0])/ np.array([0])


array([nan])

In [32]:
[dice[2][1:3] for dice in dices]

[array([0.76322064, 0.27407407]),
 array([0.67503186, 0.77113887]),
 array([0.85274497, 0.56079519]),
 array([0.86177531, 0.72733588]),
 array([0.71877072, 0.        ]),
 array([0.80283708, 0.55477802]),
 array([0.79677976, 0.70018549]),
 array([0.77026919, 0.60691004]),
 array([0.68231362, 0.75468975]),
 array([0.09860845, 0.        ]),
 array([0.82992992, 0.        ]),
 array([0.52189053, 0.5233927 ]),
 array([0.73430717, 0.25739531]),
 array([0.6060253 , 0.10350022]),
 array([0.77607293, 0.        ])]

In [48]:
data = {      "BRATS_L2_DCS": {
        "0": 0.03198548718193536,
        "1": 0.1406060606060606,
        "2": 0.9380432437098096,
        "3": 0.7744748891634566,
        "4": 0.045050385299347954,
        "5": 0.8680335764414476,
        "6": 0.8767171231078834,
        "7": 0.7641891891891892,
        "8": 0.8607803551563594,
        "9": 0.6054653773096184,
        "10": 0.7636292391895138,
        "11": 0.6623392234643587,
        "12": 0.6487541492741441,
        "13": 0.006505576208178439,
        "14": 100,
        "15": 100,
        "16": 0.6349537456920007,
        "17": 0.7702456878602582,
        "18": 0.6026694819395219,
        "19": 0.5278906300380474,
        "20": 0.314596554850408,
        "21": 0.3234369542438002,
        "22": 0.9043699530028821,
        "23": 0.02370771569745344,
        "24": 0.37958831439026514,
        "25": 0.0050920485703094395,
        "26": 100,
        "27": 0.017631737126828292,
        "28": 0.7264546566486109,
        "29": 100,
        "30": 100,
        "31": 0.3576768743400211,
        "32": 100,
        "33": 100,
        "34": 100,
        "35": 100,
        "36": 100,
        "37": 100,
        "38": 100,
        "39": 0.3629142669638715,
        "40": 0.837253516178762,
        "41": 0.2627399982229833,
        "42": 100,
        "43": 0.00398406374501992,
        "44": 0.16282894736842105,
        "45": 0.8894568690095846,
        "46": 0.6692753643121909,
        "47": 0.725327593846933,
        "48": 0.09005037783375315,
        "49": 0.8245385789619761,
        "50": 0.8773709073471345,
        "51": 0.8759251307181287,
        "52": 0.7817009517961314,
        "53": 0.5301571211353269,
        "54": 100,
        "55": 0.8101964949548592,
        "56": 0.8120355796332223,
        "57": 0.8535418147358446,
        "58": 0.7133572281959378,
        "59": 0.8073789044816659,
        "60": 0.6442209590438711,
        "61": 0.21007604562737642,
        "62": 0.1967589272909795,
        "63": 0.5767856648295344,
        "64": 0.6697213519742744,
        "65": 0.6305643594057727,
        "66": 0.27236528803249493,
        "67": 0.0018426216821293337,
        "68": 0.02298459759953759,
        "69": 0.03226394156715306,
        "70": 100,
        "71": 0.20778870772133134,
        "72": 0.22084058691867695,
        "73": 0.0,
        "74": 100,
        "75": 0.729402953135031,
        "76": 0.7181052826348413,
        "77": 0.5286025834591511,
        "78": 0.09453993933265925,
        "79": 0.5546142628984099,
        "80": 0.7302790066138145,
        "81": 0.789804604372219,
        "82": 0.6285336856010568,
        "83": 0.47619047619047616,
        "84": 100,
        "85": 0.6923812559770481,
        "86": 0.058602554470323066,
        "87": 0.7014999175869457,
        "88": 0.05001964122037449,
        "89": 0.20813451488298115,
        "90": 0.05021008403361345,
        "91": 0.14803811084929822,
        "92": 0.3245109321058688,
        "93": 100,
        "94": 0.014150943396226415,
        "95": 100,
        "96": 100,
        "97": 100,
        "98": 100,
        "99": 0.0013762916835882858,
        "100": 0.002303338744351336,
        "101": 0.0,
        "102": 0.0,
        "103": 0.0,
        "104": 0.9156817581089756,
        "105": 0.04945054945054945,
        "106": 0.7438468550592525,
        "107": 0.28042105263157896,
        "108": 0.7367861320855046,
        "109": 0.6219971592663497,
        "110": 0.7381296563994217,
        "111": 0.0,
        "112": 0.4731800766283525,
        "113": 0.7889291598023064,
        "114": 0.7061358014840821,
        "115": 0.43162341581170793,
        "116": 0.0409375,
        "117": 0.706043204812688,
        "118": 0.687084582301577,
        "119": 0.613965744400527,
        "120": 100,
        "121": 100,
        "122": 100,
        "123": 0.34995625546806647,
        "124": 0.507738392411383,
        "125": 0.16150802885734233,
        "126": 100,
        "127": 0.0370722433460076,
        "128": 0.8417687799680341,
        "129": 0.3626968914076173,
        "130": 0.5257378174330817,
        "131": 0.19166666666666668,
        "132": 100,
        "133": 100,
        "134": 0.7414387824046087,
        "135": 0.04518881339864118,
        "136": 0.15334773218142547,
        "137": 0.22875633089476646,
        "138": 0.75115562403698,
        "139": 0.8039363621453174,
        "140": 100,
        "141": 0.5795180722891566,
        "142": 0.7695088882193432,
        "143": 0.011412268188302425,
        "144": 0.28636052090472924,
        "145": 0.8264593908629442,
        "146": 0.25617977528089886,
        "147": 0.43505674653215637,
        "148": 0.8270896815438848,
        "149": 0.7449808256259869,
        "150": 0.7142127793268664,
        "151": 0.5828936657722572,
        "152": 0.27579442918791686,
        "153": 0.030529595015576325,
        "154": 0.3842277853315204,
        "155": 0.3636446262213133,
        "156": 0.5454696591853699,
        "157": 0.49838579499596447,
        "158": 0.007537688442211055,
        "159": 0.038771151634631185,
        "160": 0.2650356778797146,
        "161": 0.5443843972390177,
        "162": 0.6250925079448,
        "163": 0.8194186394965538,
        "164": 0.719562548822676,
        "165": 0.636680208937899,
        "166": 0.6789762791367498,
        "167": 0.04037520391517129,
        "168": 0.24881676808654496,
        "169": 0.8832911372185706,
        "170": 0.0,
        "171": 0.0500467037018829,
        "172": 0.533079138441569,
        "173": 0.42409738508910483,
        "174": 0.411667732059348,
        "175": 0.7693522906793049,
        "176": 100,
        "177": 0.009159606136936112,
        "178": 0.6809653916211293,
        "179": 0.26914429308292553,
        "180": 0.5749599450675211,
        "181": 0.6545529810342386,
        "182": 0.5707501047632351,
        "183": 0.3164295797589315,
        "184": 0.8631521935367394,
        "185": 100,
        "186": 100,
        "187": 0.0,
        "188": 0.2437216961712639,
        "189": 0.14910858995137763,
        "190": 0.0,
        "191": 0.6798726766327516,
        "192": 0.2980849292256453,
        "193": 0.776182023559787,
        "194": 0.8487158908507223,
        "195": 0.9559006211180124,
        "196": 0.6376804305185334,
        "197": 0.8172572241554996,
        "198": 0.7925331155870078,
        "199": 0.8826247954760743,
        "200": 0.8312610105445911,
        "201": 0.6167539267015707,
        "202": 0.2336734693877551,
        "203": 0.8181946142163944,
        "204": 0.7808321830013804,
        "205": 0.5029382699285612,
        "206": 0.445214807478024,
        "207": 0.7585353140474329,
        "208": 0.017766497461928935,
        "209": 0.9164527182529535,
        "210": 0.882788406209389,
        "211": 0.9480180590647392,
        "212": 0.802530880773362,
        "213": 0.8976147268342065,
        "214": 0.7792588330159643,
        "215": 0.7680970957954053,
        "216": 0.6836719450253959,
        "217": 0.7176014239417943,
        "218": 0.9296134253808114,
        "219": 0.017754569190600523,
        "220": 0.7627072336426128,
        "221": 0.8564055510953091,
        "222": 0.3629032258064516,
        "223": 0.9048857185640969,
        "224": 0.8766493454668483,
        "225": 0.9633727670782634,
        "226": 0.7866003899470786,
        "227": 0.812160751565762,
        "228": 0.8606414604025118,
        "229": 0.9140619871886451,
        "230": 0.5454796030871003,
        "231": 0.5375052111250075,
        "232": 0.8025894606086665,
        "233": 0.7183073496659242,
        "234": 0.1951219512195122,
        "235": 0.8810014662594948,
        "236": 0.38138440860215056,
        "237": 0.9388659857885687,
        "238": 0.8291506450365179,
        "239": 0.8589371521004805,
        "240": 0.41895845523698066,
        "241": 0.856326478585996,
        "242": 0.9119424265055333,
        "243": 0.02405097071185536,
        "244": 0.7139110337843102,
        "245": 0.6710570753123944,
        "246": 0.2214435879467414,
        "247": 0.6953039209353401,
        "248": 0.9096012611989079,
        "249": 0.010869565217391304,
        "250": 0.927424912799415,
        "251": 0.8030898892208326,
        "252": 0.3421686746987952,
        "253": 0.8889284500477431,
        "254": 0.9386877166604017,
        "255": 0.7202495260986458,
        "256": 0.7507747424988804,
        "257": 0.7021828749070758,
        "258": 0.8997216074474764,
        "259": 0.633986013986014,
        "260": 0.7417340403952593,
        "261": 0.7892695334234162,
        "262": 0.8096270045636489,
        "263": 0.7617439897549334,
        "264": 0.09245742092457421,
        "265": 0.8724765556428142
      },}["BRATS_L2_DCS"]

In [52]:
np.mean([data[key] if data[key] != 100 else 0 for key in data ]), np.mean([data[key] for key in data if data[key] != 100])

(0.46234065548629544, 0.52556672803143)